In [1]:
import torch
import sklearn.datasets as skds
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
import torchvision
from torch import nn

# Define model and custom layers

In [2]:
class MyCustomLayer(nn.Module):
	def __init__(self, size_in, size_out):
		super().__init__()

		self.size_in, self.size_out = size_in, size_out
		weights = torch.Tensor(size_out, size_in)
		self.weights = nn.Parameter(weights)  # nn.Parameter is a Tensor that's a module parameter.
		bias = torch.Tensor(size_out)
		self.bias = nn.Parameter(bias)

		self.linear1 = nn.Linear(size_in, size_out)
		self.activation = nn.ReLU(inplace=True)

		# initialize weights and biases
		
	
	def forward(self, x):
		# 1. X is batch of the samples, as input
		#x = torch.flatten(x)

		# 2. Split input into two tensors with same length (we can assume that the length of X is even)
		assert x.shape[0] % 2 == 0
		x1, x2 = torch.tensor_split(x, 2, dim=1)
		assert x1.size() == x2.size()

		# 3. Put two tensors into the same aggregation layer (linear, for example)
		x1 = self.linear1(x1)
		x2 = self.linear1(x2)

		# 3. Then put into activation layer
		x1 = self.activation(x1)
		x2 = self.activation(x2)

		# 4. Concatinate two halves to create output Y
		Y = torch.cat((x1, x2))
		return Y

model = nn.Sequential(
	MyCustomLayer(392, 20),
	nn.Softmax()
)


# Load train data

In [3]:
# train_data_transformed = torchvision.datasets.FashionMNIST( 
#     root="/22961", train=True, download=False, 
#     transform=torchvision.transforms.PILToTensor())
# train_dataloader = DataLoader(train_data_transformed, batch_size=4)

# trans=torchvision.transforms.Compose(
#     [torchvision.transforms.PILToTensor(),
#      torchvision.transforms.ConvertImageDtype(torch.float)])
# train_data_transformed = torchvision.datasets.FashionMNIST(
#     root="/22961", train=True, download=True,
#     transform=trans)

dataset = torchvision.datasets.FashionMNIST(root="/22961", train=True, download=True, transform=torchvision.transforms.PILToTensor())
dataloader = DataLoader(dataset, batch_size=10)

# Define cost and optimizer functions

We use Negative Log Likelihood Loss (NLLLoss), which is our Cross Entropy function

And we use classic SGD optimizer to find minimum

In [4]:
CE_loss=torch.nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# Define batch iterative function

This function is called for each batch

In [5]:
def iterate_batch(idx):
	print(f"Batch: {idx}")
	imgs, labels = next(iter(dataloader))
	imgs = imgs.flatten(start_dim=1)
	optimizer.zero_grad()
	y_model=model(imgs.float())

	loss=CE_loss(y_model,labels)
	loss.backward()
	optimizer.step()

	predicted_labels = y_model.argmax(dim=1)
	acc = (predicted_labels == labels).sum()/len(labels)
	return loss.detach(), acc.detach()

# Train

In [6]:
from tqdm import tqdm
batches=len(dataloader)
print("num of batches: \n", batches)
batch_loss=torch.zeros(batches)
batch_acc=torch.zeros(batches)
for idx in tqdm(range(batches)):
	batch_loss[idx], batch_acc[idx] = iterate_batch(idx)

num of batches: 
 6000


  0%|          | 0/6000 [00:00<?, ?it/s]

Batch: 0


C:\Users\Shlomi\anaconda3\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
  0%|          | 0/6000 [00:00<?, ?it/s]


ValueError: Expected input batch_size (20) to match target batch_size (10).

# Help is here

For split: `torch.split`

 torch.mean(batch)

שאלת המשך. איך איתחלתם? XAVIER?

ANSWER: I used reset_parameters

שאלה 2 שכבות

השתמשתי רק ב

רק ב-nn.Linear ו-DropNorm